In [ ]:
import polars as pl
#pl.Config.set_tbl_rows(2000)

In [40]:
from src.pool import PG

NEW_TANKS_QUERY = """--sql 
    SELECT DISTINCT source_key, division_id, source_id
    FROM sdm_dba.data_catalog dc
    WHERE source_key NOT LIKE '%FAC%'
    ORDER BY source_key;
"""

new_tanks_df = await PG.fetch(NEW_TANKS_QUERY)
if new_tanks_df is None:
    raise ValueError("df is None")
if new_tanks_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

new_tanks_df

source_key,division_id,source_id
str,i64,str
"""1100901""",26,"""11009"""
"""1100902""",26,"""11009"""
"""1100903""",26,"""11009"""
"""1100904""",26,"""11009"""
"""1100905""",26,"""11009"""
…,…,…
"""9884004""",40,"""98840"""
"""9884005""",40,"""98840"""
"""9884006""",40,"""98840"""


In [41]:
pattern = r'0\d$'
oil_tanks_df = new_tanks_df.filter(pl.col("source_key").str.contains(pattern)) 
new_oil_df = oil_tanks_df.group_by("source_id").agg(pl.count("source_key").alias("oil_tank_count"))
new_oil_df = new_oil_df.sort("source_id")
new_oil_df

source_id,oil_tank_count
str,u32
"""11009""",6
"""48001""",6
"""69419""",4
"""98743""",3
"""98750""",3
…,…
"""98836""",6
"""98837""",5
"""98838""",5


In [42]:
water_tanks_df = new_tanks_df.filter(pl.col("source_key").str.contains("W"))
new_df = water_tanks_df.group_by("source_id").agg(pl.count("source_key").alias("water_tank_count"))
new_df = new_df.sort("source_id")
new_df

source_id,water_tank_count
str,u32
"""11009""",3
"""48001""",2
"""69419""",3
"""98743""",2
"""98750""",2
…,…
"""98836""",3
"""98837""",3
"""98838""",2


In [43]:
combined_count_df = new_df.join(new_oil_df, on="source_id")
combined_count_df

source_id,water_tank_count,oil_tank_count
str,u32,u32
"""11009""",3,6
"""48001""",2,6
"""69419""",3,4
"""98743""",2,3
"""98750""",2,3
…,…,…
"""98836""",3,6
"""98837""",3,5
"""98838""",2,5


In [44]:
from src.pool import PG

DATA_CATALOG_QUERY = """--sql 
    SELECT *
    FROM sdm_dba.data_catalog AS dc
"""

dc_df = await PG.fetch(DATA_CATALOG_QUERY)
if dc_df is None:
    raise ValueError("df is None")
if dc_df.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

dc_df

key_metric,source_key,source_id,metric_nice_name,uom,division_id
object,str,str,str,str,i64
d0613649-4688-445b-aca9-b83858a7476d,"""6941902""","""69419""","""OilTank2Level""","""in""",5
a5de3be7-73fe-489b-b861-2ab88c3b0029,"""6941903""","""69419""","""OilTank3Level""","""in""",5
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,"""6941901""","""69419""","""OilTank1Volume""","""bbl""",5
a99b4283-d21f-4840-b1ae-d91a55011b27,"""6941902""","""69419""","""OilTank2Volume""","""bbl""",5
c2650a39-a755-4855-976b-e3f712197666,"""6941903""","""69419""","""OilTank3Volume""","""bbl""",5
…,…,…,…,…,…
92ee60bc-576f-4288-8ac2-14d1755a18e2,"""1100906""","""11009""","""OilTank6Volume""","""bbl""",26
b2d46e76-51ba-4a4e-aeeb-4333695f5b3f,"""11009W3""","""11009""","""WaterTank3Level""","""in""",26
3d332f5e-1760-467c-902e-a0bcc8c91ece,"""11009W3""","""11009""","""WaterTank3Volume""","""bbl""",26


In [45]:
new_new_df = dc_df.join(combined_count_df, on="source_id")
new_new_df

key_metric,source_key,source_id,metric_nice_name,uom,division_id,water_tank_count,oil_tank_count
object,str,str,str,str,i64,u32,u32
d0613649-4688-445b-aca9-b83858a7476d,"""6941902""","""69419""","""OilTank2Level""","""in""",5,3,4
a5de3be7-73fe-489b-b861-2ab88c3b0029,"""6941903""","""69419""","""OilTank3Level""","""in""",5,3,4
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,"""6941901""","""69419""","""OilTank1Volume""","""bbl""",5,3,4
a99b4283-d21f-4840-b1ae-d91a55011b27,"""6941902""","""69419""","""OilTank2Volume""","""bbl""",5,3,4
c2650a39-a755-4855-976b-e3f712197666,"""6941903""","""69419""","""OilTank3Volume""","""bbl""",5,3,4
…,…,…,…,…,…,…,…
92ee60bc-576f-4288-8ac2-14d1755a18e2,"""1100906""","""11009""","""OilTank6Volume""","""bbl""",26,3,6
b2d46e76-51ba-4a4e-aeeb-4333695f5b3f,"""11009W3""","""11009""","""WaterTank3Level""","""in""",26,3,6
3d332f5e-1760-467c-902e-a0bcc8c91ece,"""11009W3""","""11009""","""WaterTank3Volume""","""bbl""",26,3,6


In [46]:
import polars as pl
import random
from datetime import datetime, timedelta

def tank_timeseries_with_value() -> pl.DataFrame:
    time_series = []
    value = 0 

    for tank in new_new_df.iter_rows(named=True):
        key_metric = tank["key_metric"]
        identifier = tank["metric_nice_name"]

        # Generate a consistent timestamp for this iteration
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)
        create_ts = start_date + (end_date - start_date) * random.random()

        if "ID" in identifier:
            if "Oil" in identifier:
                value = random.randint(1, tank["oil_tank_count"])
            else:
                value = random.randint(1, tank["water_tank_count"])
        else: 
            continue

        time_series.append({
            "key_metric": key_metric,
            "ts": create_ts,  # Use the same timestamp for each metric
            "value": value
        })

    time_series_df = pl.DataFrame(time_series)
    return time_series_df


In [47]:
tank_timeseries_with_value()

key_metric,ts,value
object,datetime[μs],i64
7eb0b21d-a87f-4a27-8751-f7318db2fdc8,2023-11-22 12:05:15.044649,2
de925327-bbae-4c3d-b93c-c65094656a58,2023-10-13 03:04:53.057759,1
a08a596d-c3f1-462d-aecf-88abb45cabc1,2024-01-13 18:28:47.301841,2
2b9672ff-5a41-4954-bb31-174d434ae296,2023-11-25 20:48:55.422467,2
c40c4067-8262-434c-9f83-5ccad23c28e8,2023-10-30 05:48:38.080551,2
…,…,…
bf404389-f86f-47fe-8eb6-aac06db9bd77,2023-09-25 00:28:07.009832,1
6238907c-5216-423d-88a3-0c261c5929d1,2024-06-01 14:39:04.925983,5
b4fa08d8-bbb4-4957-907e-591368c5bbc1,2024-02-10 18:49:29.787846,1


In [48]:
from src.pool import PG

await PG.insert(tank_timeseries_with_value(), "sdm_dba.timeseries_data_two")

In [ ]:
from src.pool import PG

TS_QUERY = """--sql 
    SELECT *
    FROM sdm_dba.data_catalog dc
    WHERE metric_nice_name NOT LIKE '%TankID%'
    ORDER BY source_key;
"""

ts_query = await PG.fetch(TS_QUERY)
if ts_query is None:
    raise ValueError("df is None")
if ts_query.shape[0] == 0:
        raise ValueError("The fetched DataFrame is empty.")

ts_query

In [ ]:
import polars as pl
import random
from datetime import datetime, timedelta
from perlin_noise import PerlinNoise

# Initialize Perlin noise generator
random_noise = PerlinNoise(octaves=1)

# Function to create the initial tank time series data
def tank_timeseries_with_value2(num_points: int = 40) -> pl.DataFrame:
    time_series = []

    for tank in ts_query.iter_rows(named=True):
        key_metric = tank["key_metric"]
        identifier = tank["metric_nice_name"]

        # Generate  points per key_metric
        for _ in range(num_points):
            # Keep timestamps within the last 7 days
            end_date = datetime.now()
            start_date = end_date - timedelta(days=7)
            create_ts = start_date + (end_date - start_date) * random.random()

            # Generate value using Perlin noise
            if "Volume" in identifier:
                value = 300 + random_noise.noise(create_ts.timestamp() / 100000) * 180
            elif "Level" in identifier:
                value = 79 + random_noise.noise(create_ts.timestamp() / 100000) * 19
            else:
                value = 230 + random_noise.noise(create_ts.timestamp() / 100000) * 170

            time_series.append({
                "key_metric": key_metric,
                "ts": create_ts,
                "value": value
            })

    time_series_df = pl.DataFrame(time_series)
    return time_series_df

# Generate the time series data with Perlin noise
initial_df = tank_timeseries_with_value2(num_points=40)

In [ ]:
tank_timeseries_with_value2()

In [ ]:
from src.pool import PG

await PG.insert(tank_timeseries_with_value2(), "sdm_dba.timeseries_data_two")